# Conversión de Imágenes en Archivo CSV
---
**Montar la carpeta de Google Drive y definir constantes para trabajar**

In [1]:
MNT_DRIVE_DIR = '/content/drive/'
DRIVE_DIR = '/content/drive/MyDrive/'
DATOS_DIR = DRIVE_DIR + 'Datos/'
FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/Fuentes'

from google.colab import drive
drive.mount(MNT_DRIVE_DIR)

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Mounted at /content/drive/


**Funciones auxiliares para convertir imagenes en carpetas a caracteristicas geométricas en un archivo separado por comas (csv)**

---



In [2]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.measure import find_contours, regionprops
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square
from skimage.segmentation import clear_border
from skimage import io
import pandas as pd
import numpy as np
import glob
import math

# funcion para extraer características geométricas de un objeto dentro de una imagen
def extraer_caracteristicas(imagen):
    # busca umbral global con método estadístico de Otsu
    umbral = threshold_otsu(imagen)

    # binariza la imagen
    imagen_bn = (imagen > umbral)*1

    # cierra pequeños huecos/cortes que pudiera tener la imagen de la mano
    imagen_bn = closing(imagen_bn, square(3))

    # remueve artefactos que pudiera tener la imagen en los bordes
    imagen_lista = clear_border(imagen_bn)

    # extrae regiones de pixeles conectados, para los ejemplos debería haber
    # solo una única region. Para cada region obtiene distintas propiedades
    # que caracterizan a dicha region, ej: area, perimetro, centro, eje mayor y
    # menor, orientacion, coordenadas y área del recuadro que la contiene
    regiones = regionprops(imagen_lista)

    return (regiones[0], imagen_lista)


def mostrar_imagen(axs, imagen, cmap='gray'):
    # remueve ejes de los graficos
    axs.get_xaxis().set_ticks([])
    axs.get_yaxis().set_ticks([])

    axs.imshow(imagen, cmap=cmap)  


# funcion para procesar todas las imagenes de manos mostrando dedos de una carpeta 
# y generar un archivo csv con las características geometricas de lad mismas
def convertir_dataset(dir_orig, dir_arch_dest):

    # definición de columnas a guardar en el archivo csv
    columnas=['AreaNorm', 'PerimNorm', 'RazonEjes', 'Excentricidad', 'Solidez',
                               'Extension', 'CantDedos']
    
    # recupera nombrs de archivos
    archivos = glob.glob(dir_orig+'*.png')
    
    data = np.empty((0, len(columnas)))

    # recorre la lista de archivos para procesar cada imagen. Guarda las características
    # en un arreglo temporal
    for nro, archivo in enumerate(archivos):
        
        print('\rTransformando %d imágenes: %6.2f%%' % (len(archivos), 100*(nro+1)/len(archivos)), end='')
        imagen = io.imread(archivo)
        
        # el nombre de archivo tiene la cantidad de dedos que se estan mostrando
        cant_dedos = int(archivo[-6])

        # extrae caracteristicas
        props, imagen_bn = extraer_caracteristicas(imagen)
        

        area = props.filled_area                # area en pixeles (incluyendo huecos)
        ej_mayor = props.major_axis_length      # largo en pixeles del eje mayor en elipse de m. a.
        ej_menor = props.minor_axis_length      # largo en pixeles del eje menor en elipse de m. a.
        perim = props.perimeter                 # perimetro en pixeles
        # EXCENTRICIDAD: relacion con elipse que contiene la region. 
        # cercano a 0 es un circulo, cercano a 1 es mas estirada la elipse
        excentr = props.eccentricity      
        # SOLIDEZ: razon entre la cantidad de pixeles de la region original y de la 
        # region convexa. Para generar una region convexa se completan los pixeles
        # de forma de eliminar regiones convexas de una figura. La región convexa
        # de una estrella de 5 puntas se convertira en un pentágono al completarla
        solidez = props.solidity
        # EXTENSION: razon entre pixeles de la region original y el rectangulo que
        # la contiene (bounding box)
        extension = props.extent     
        area = area /  ((ej_mayor*ej_menor))
        perim = perim / ((ej_mayor+ej_menor)/2)
        #perim = perim / math.sqrt(ej_mayor*ej_menor)
        razon_ej = ej_menor/ej_mayor
        data = np.append(data, np.array([[area, perim, razon_ej, excentr, 
                                          solidez, extension, cant_dedos]]), axis=0)

    df = pd.DataFrame(columns=columnas, data = data)
    df.to_csv(dir_arch_dest)
    

**Asignar atributos sin valor o con valor nulo**

In [3]:
DEDOS_DIR = DATOS_DIR + 'Fingers/'
convertir_dataset(DEDOS_DIR+'train/', DEDOS_DIR+'fingers_train.csv')       
convertir_dataset(DEDOS_DIR+'test/' , DEDOS_DIR+'fingers_test.csv')       


Transformando 17999 imágenes:   1.03%

KeyboardInterrupt: ignored

**Normalización de valores de atributos y valores a predecir**

In [ ]:
# Imprime matriz de correclacion
#corr = df.corr()
#plt.figure()
#sns.heatmap(corr, square=True,  annot=True, linewidths=.5)

datos = np.array(df)
normalizarEntrada = 1

attPred = [12,14] #12 mpg-highway y 14 precio  #-- nro de atributos a predecir    
T = datos[:, attPred]
X = np.delete(datos, attPred, 1) # elimina columnas de atributos a predecir

if normalizarEntrada:
    data_scaler, target_scaler = StandardScaler(), StandardScaler()
    X = data_scaler.fit_transform(X)
    T = target_scaler.fit_transform(T)
    
# %% Separa los datos y clase en grupo de entrenamiento y validacion
x_train, x_val, y_train, y_val = train_test_split(X, T, test_size = 0.2,  shuffle = True)

**Construccion del modelo**

In [ ]:
6# cantidad de pasadas de los datos
EPOCAS = 1000
# cantidad de datos a procesar para actualizar pesos
TAM_LOTE = 64

ENTRADAS = X.shape[1]
SALIDAS = len(attPred)

ACTIVACION = 'LeakyReLU'
#ACTIVACION = 'ReLU'
#ACTIVACION = 'tanh'
#ACTIVACION = 'sigmoid'

#OPTIMIZADOR = 'sgd'
OPTIMIZADOR = 'rmsprop'
#OPTIMIZADOR = 'adam'
PACIENCIA = 15

model = Sequential()
model.add(Dense(6, activation= ACTIVACION, input_shape=[ENTRADAS]))
model.add(Dense(3, activation= ACTIVACION))
model.add(Dense(SALIDAS))

model.summary()

# obtiene la arquitectura para el modelo y lo compila
model.compile(optimizer=OPTIMIZADOR, loss='mae', metrics = ['accuracy', 'mae', 'mse'])

**Entrenamiento del modelo con parada temprana**

In [ ]:
# El parámetro patience indica la cantidad de epocas que deben transcurrir 
# sin mejoras en el entrenamiento
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=PACIENCIA)

# Entrena el modelo y guarda la historia del progreso    
history = model.fit(x = x_train, y = y_train, batch_size = TAM_LOTE, epochs = EPOCAS, 
              validation_data = (x_val, y_val), callbacks=[early_stop])

print('\n'+'-'*80)
print('Epocas utilizadas: %d' % len(history.epoch))

# %% Evalua e informa resultado de entrenamiento, validación y testeo
# evalua el modelo con los datos de entreanmiento
pred = model.evaluate(x_train, y_train, verbose=0)
print('\nEfectividad del modelo con datos de entrenamiento para:' )
print(" - Accuracy: %6.2f%%" % (pred[1]*100))
print(" - Pérdida : %9.5f" % (pred[0]))

# evalua el modelo con los datos de validacion
pred = model.evaluate(x_val, y_val, verbose=0)
print('\nEfectividad del modelo con datos de validacion para:' )
print(" - Accuracy: %6.2f%%" % (pred[1]*100))
print(" - Pérdida : %9.5f" % (pred[0]))


**Gráficos con evolución de curvas de error y accuracy**

In [ ]:
hist = history.history

def dibujar_curva(axs, value, epocas, hist):
    axs.plot(epocas, hist[value], label='Testeo')
    axs.plot(history.epoch, hist['val_'+value], label = 'Validacion')

    axs.set(xlabel="Epocas", ylabel=value)
    axs.legend()
    
fig, axs = plt.subplots(1,3, figsize=(20,6))

dibujar_curva(axs[0], 'mae', history.epoch, hist)
dibujar_curva(axs[1], 'mse', history.epoch, hist)
dibujar_curva(axs[2], 'accuracy', history.epoch, hist)

**Retroalimentación visual de las predicciones sobre los datos de validación**

In [ ]:
y_pred = model.predict(x_val)

y_val_no_norm = target_scaler.inverse_transform(y_val)
y_pred_no_norm = target_scaler.inverse_transform(y_pred)

fig, axs = plt.subplots(1,2, figsize=(20,6))

# lista con nro de iteracion
sample = list(range(len(y_val_no_norm)))

axs[0].plot(sample, y_val_no_norm[:,0],  "o", color='blue')
axs[0].plot(sample, y_pred_no_norm[:,0], "o", color='red')
axs[0].set(xlabel='Ejemplo', ylabel='Millas x galón')

axs[1].plot(sample, y_val_no_norm[:,1],  "o", color='blue')
axs[1].plot(sample, y_pred_no_norm[:,1], "o", color='red')
axs[1].set(xlabel='Ejemplo', ylabel='Precio del Vehículo')